In [17]:
#importing libraries
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from collections import Counter
from sklearn.linear_model import LinearRegression
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer
import spacy

%matplotlib inline
sns.set(color_codes = True)

In [18]:
df=pd.read_excel('beer-reviews.xlsx')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
!python -m spacy download en_core_web_md 

                                              0.0/42.8 MB ? eta -:--:--
                                              0.1/42.8 MB 3.3 MB/s eta 0:00:13
                                              0.3/42.8 MB 4.2 MB/s eta 0:00:11
                                              0.8/42.8 MB 6.0 MB/s eta 0:00:08
     -                                        1.8/42.8 MB 10.5 MB/s eta 0:00:04
     --                                       2.5/42.8 MB 12.5 MB/s eta 0:00:04
     ----                                     4.6/42.8 MB 18.4 MB/s eta 0:00:03
     -----                                    6.3/42.8 MB 22.5 MB/s eta 0:00:02
     -------                                  8.3/42.8 MB 24.1 MB/s eta 0:00:02
     ---------                               10.9/42.8 MB 38.5 MB/s eta 0:00:01
     -----------                             12.9/42.8 MB 50.4 MB/s eta 0:00:01
     -------------                           14.7/42.8 MB 50.4 MB/s eta 0:00:01
     --------------                          16.4/4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
stop = stopwords.words('english')
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)
df['body']=df['body'].str.lower()

df['body'] = df['body'].astype(str).apply(remove_stopwords)
df['body']


0                     amazing stout expensive worth price
1           beer meet exceeded unreal hype simply amazing
2                   let sit warm room little unbelievable
3       small pour ebony dark real head intense rich a...
4       vintage 2022 served tap toppling goliath snuli...
                              ...                        
6626    poured bottle snifter home thanks darin share ...
6627    good god man holy shit really good beer blesse...
6628    appearance bubble show lifts head fullsome thr...
6629    pours sure describe color golden yellow brown ...
6630    tropical nose sweet like sunnyd nice acidity a...
Name: body, Length: 6631, dtype: object

In [21]:
df.head()

,item,body,rating
0,Kentucky Brunch Brand Stout,amazing stout expensive worth price,5.00
1,Kentucky Brunch Brand Stout,beer meet exceeded unreal hype simply amazing,5.00
2,Kentucky Brunch Brand Stout,let sit warm room little unbelievable,5.00
3,Kentucky Brunch Brand Stout,small pour ebony dark real head intense rich a...,4.67
4,Kentucky Brunch Brand Stout,vintage 2022 served tap toppling goliath snuli...,5.00


# Task B
Assume that a customer, who will be using this recommender system, has specified 3 attributes
in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes
from the actual data)
Perform a word frequency analysis of beer reviews is a better way
to find important attributes in the actual data.

In [22]:
# Calculate word frequencies using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['body'])

# Calculate the frequency of each word in the reviews
word_frequencies = pd.DataFrame({'word': vectorizer.get_feature_names_out(), 'frequency': np.array(X.sum(axis=0))[0]})
word_frequencies=word_frequencies.sort_values(['frequency'],ascending=False)
word_frequencies

,word,frequency
1829,beer,5266
7320,head,3650
14880,taste,3483
8867,light,2891
8885,like,2537
...,...,...
7520,hoax,1
7521,hobbyists,1
7526,hog,1
7527,hogan,1


In [23]:
word_list = list(word_frequencies['word'])
attributes = [x for x in word_list if x in beer_attributes]
print(attributes)

NameError: name 'beer_attributes' is not defined

In [24]:
array_df = pd.DataFrame({'attribute': attributes})
attr_word_freq=word_frequencies.merge(array_df,left_on='word', right_on='attribute')
attr_word_freq= attr_word_freq.drop('attribute',axis=1)
attr_word_freq

NameError: name 'attributes' is not defined

# Task F

How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? Explain. This article may be useful: https://medium.com/swlh/word-embeddings-versus-bag-of-words-the-curious-case-of-recommender-systems-6ac1604d4424?source=friends_link&sk=d746da9f094d1222a35519387afc6338
Note that the article doesn’t claim that bag-of-words will always be better than word embeddings for recommender systems. It lays out conditions under which it is likely to be the case. That is, depending on the attributes you use, you may or may not see the same effect. 


In [25]:
!pip install -q transformers
from transformers import pipeline


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
cust_attr= ['robust','fruity','hoppy']

# Load spaCy with medium-sized pretrained word vectors
nlp = spacy.load('en_core_web_md')

# Function to calculate word vector similarity
def word_vector_similarity(df):
    text1 = df['body']
    text2 = df['Customer attribute']
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

sentiment_pipeline = pipeline("sentiment-analysis")
def sentiment_analysis(df):
    sentiment= sentiment_pipeline(df['body'])
    if sentiment[0]['label']=='POSITIVE':
        return sentiment[0]['score']
    else:
        return (-sentiment[0]['score'])

df1= df.copy()
df2= df.copy()
df3=df.copy()

df1['Customer attribute'] = cust_attr[0]
df2['Customer attribute'] = cust_attr[1]
df3['Customer attribute'] = cust_attr[2]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [27]:
df1['Similarity Score'] = df1.apply(word_vector_similarity, axis=1)
df2['Similarity Score'] = df2.apply(word_vector_similarity, axis=1)
df3['Similarity Score'] = df3.apply(word_vector_similarity, axis=1)

df1['Sentiment score'] = df1.apply(sentiment_analysis, axis=1)
df2['Sentiment score'] = df2.apply(sentiment_analysis, axis=1)
df3['Sentiment score'] = df3.apply(sentiment_analysis, axis=1)

C:\Users\rahul\AppData\Local\Temp\ipykernel_20132\1809566172.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
C:\Users\rahul\AppData\Local\Temp\ipykernel_20132\1809566172.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
C:\Users\rahul\AppData\Local\Temp\ipykernel_20132\1809566172.py:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
Token indices sequence length is longer than the specified maximum sequence length for this model (580 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (580) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
df1.head()

,item,body,rating,Customer attribute,Similarity Score
0,Kentucky Brunch Brand Stout,amazing stout expensive worth price,5.00,robust,0.486838
1,Kentucky Brunch Brand Stout,beer meet exceeded unreal hype simply amazing,5.00,robust,0.410720
2,Kentucky Brunch Brand Stout,let sit warm room little unbelievable,5.00,robust,0.285641
3,Kentucky Brunch Brand Stout,small pour ebony dark real head intense rich a...,4.67,robust,0.445239
4,Kentucky Brunch Brand Stout,vintage 2022 served tap toppling goliath snuli...,5.00,robust,0.492351


In [ ]:
df['Overall score']= df['Similarity score']*df['Sentiment score']

# Task H
Using the top four attributes of beer (from word frequency analysis), calculate the lifts between
these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among
the remaining 9) using the lift values. Explain your method.

In [11]:
def initialize(list1, list2):
    attribute_mentions_per_review = {attribute: set() for attribute in list1 + list2}
    co_mentions = {}
    for attr1 in list1 + list2:
        co_mentions[attr1] = {}
        for brand in list1 + list2:
            co_mentions[attr1][brand] = 0
    individual_mentions = {attr: 0 for attr in list1 + list2}
    lift_ratios = {}
    lift_already_calculated = set()
    return co_mentions, attribute_mentions_per_review, individual_mentions, lift_ratios, lift_already_calculated

In [12]:
def calculate_lift(attr1, brand, co_mentions, individual_mentions, total_posts):
    if attr1 == brand:
        return 0  # Lift ratio between the same brand is 0
    # Calculate lift using the formula: lift(attr1, brand) = (P(attr1 and brand) / (P(attr1) * P(brand))) * N
    p_attr1_and_brand = co_mentions[attr1][brand]
    p_attr1 = individual_mentions[attr1]
    p_brand = individual_mentions[brand]
    if p_attr1 == 0 or p_brand == 0:
        return 0  
    else:
        return (p_attr1_and_brand / (p_attr1 * p_brand)) * total_posts

In [103]:
#selected_beer_brands=["Mornin' Delight","Nelson","Stickee Monkee","The Wild One","Corsendonk Christmas Ale"]
selected_beer_brands=["Old Chub","Voodoo Ranger Juicy Haze IPA","Harvest Ale (Limited Edition)","The Angel's Share - Bourbon Barrel-Aged","Weihenstephaner Korbinian","G'Knight","Mirror Pond Pale Ale","Antarctica Pilsen","Dark Lord","Breakfast Stout"]
attributes=list(attr_word_freq['word'])[:10]
attributes

['sweet',
 'carbonation',
 'mouthfeel',
 'smooth',
 'caramel',
 'clear',
 'sweetness',
 'dry',
 'bitter',
 'malty']

In [104]:
lift_df = pd.DataFrame(index=attributes, columns=selected_beer_brands)
for brand in selected_beer_brands:
    for attr in attributes:
        # Calculate support and joint support
        df['body'].fillna('', inplace=True)
        support_attr = len(df[df['body'].str.contains(attr)]) / len(df)
        support_brand = len(df[df['item'] == brand]) / len(df)
        joint_support = len(df[(df['body'].str.contains(attr)) & (df['item'] == brand)]) / len(df)
        
        # Calculate lift
        if support_attr * support_brand == 0:
            lift = 0  # Avoid division by zero
        else:
            lift = joint_support / (support_attr * support_brand)

        lift_df.at[attr, brand] = lift



In [105]:
lift_df

,Old Chub,Voodoo Ranger Juicy Haze IPA,Harvest Ale (Limited Edition),The Angel's Share - Bourbon Barrel-Aged,Weihenstephaner Korbinian,G'Knight,Mirror Pond Pale Ale,Antarctica Pilsen,Dark Lord,Breakfast Stout
sweet,2.692776,0.349064,2.094381,1.178089,1.713585,1.570786,0.523595,0.0,2.24398,1.256629
carbonation,0.576906,0.448705,1.682644,1.009586,1.468489,0.0,1.346115,2.019172,1.730719,0.807669
mouthfeel,0.841791,0.0,1.473135,2.209702,1.071371,2.209702,0.98209,0.0,1.683583,0.589254
smooth,0.979191,0.761593,2.28478,2.570377,0.623122,0.0,1.14239,1.14239,0.979191,2.056302
caramel,3.88466,0.0,3.776753,5.66513,4.120094,2.266052,1.510701,0.0,0.0,0.0
clear,0.999849,0.0,0.0,0.0,0.0,0.874868,2.332982,3.499473,0.0,0.0
sweetness,1.305588,0.0,3.046373,1.14239,0.830829,3.427169,0.0,0.0,1.305588,2.741736
dry,1.063813,0.0,0.620557,0.930836,0.676972,0.930836,0.0,0.0,0.0,0.744669
bitter,0.619514,0.481845,0.722767,0.0,0.788473,1.08415,0.722767,0.722767,0.619514,2.601961
malty,5.564719,0.0,1.082029,0.0,2.36079,3.246086,0.0,0.0,0.0,0.0


In [106]:
# Choose a reference beer (replace with the beer you want to use as reference)
reference_beer = 'Weihenstephaner Korbinian'

# Create a dictionary to store the most similar beer for each reference attribute
most_similar_beers = {}

# Iterate through the remaining nine beers
for beer in selected_beer_brands:
    if beer != reference_beer:  # Exclude the reference beer itself
        most_similar_attribute = None
        highest_lift = 0.0

        # Find the attribute with the highest lift value when associated with the reference beer
        for attribute in attributes:
            lift_value = lift_df.at[attribute, beer]
            if lift_value > highest_lift:
                highest_lift = lift_value
                most_similar_attribute = attribute

        # Store the most similar beer for the reference beer
        most_similar_beers[beer] = (most_similar_attribute, highest_lift)

# Find the beer with the highest lift value
most_similar_beer = max(most_similar_beers, key=lambda x: most_similar_beers[x][1])
similar_attribute, highest_lift = most_similar_beers[most_similar_beer]

print(f"The most similar beer to {reference_beer} is {most_similar_beer} based on the attribute '{similar_attribute}' with a lift value of {highest_lift}.")

The most similar beer to Weihenstephaner Korbinian is The Angel's Share - Bourbon Barrel-Aged based on the attribute 'caramel' with a lift value of 5.665129781420765.
